# Investigate medication usage

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

from pathlib import Path

In [2]:
# 📁 Base path = where this notebook lives
root_p = Path().resolve()

# 📂 Input paths
validation_p = root_p / '../../output_data/Results/Validation'
data_p = root_p / '../../source_data/Data'

model1_p = validation_p / 'validation_net_split_1_model_1_combined_p_values.tsv'
pheno_valid_p = data_p / 'ABIDE_2_Pheno_PSM_matched_ados.tsv'

# 📂 Output paths
fig_p = root_p / '../../output_data/Supplemental/medication_usage'
fig_p.mkdir(parents=True, exist_ok=True)

## Load and preparation

In [3]:
pheno_valid = pd.read_csv(pheno_valid_p, sep='\t')
model1 = pd.read_csv(model1_p, sep='\t')
model1.rename(columns={'V1': 'p_ASD', 'V2': 'p_TDC'}, inplace=True)

In [4]:
pheno_valid.loc[:, 'is_hps'] = ((model1.p_ASD>0.2) & (model1.p_TDC<=0.2)).values
pheno_valid.loc[:, 'p_ASD'] = model1.loc[:, 'p_ASD'].values
pheno_valid.loc[:, 'p_TDC'] = model1.loc[:, 'p_TDC'].values

In [5]:
# Compute actual indices to avoid trusting the DF indices
hps_idx = np.where(pheno_valid.is_hps)[0]
non_hps_idx = np.where(~pheno_valid.is_hps)
hps_asd_idx = np.where((pheno_valid.is_hps) & (pheno_valid.DX_GROUP=='Autism'))[0]
non_hps_asd_idx = np.where((~pheno_valid.is_hps) & (pheno_valid.DX_GROUP=='Autism'))[0]

## Calculate proportion of ASD taking medication, for those identified by HRS and those not

In [6]:
hps_asd_df = pheno_valid.iloc[hps_asd_idx]
len(hps_asd_df)

9

In [7]:
hps_asd_df[hps_asd_df.CURRENT_MED_STATUS == 'taking medication']['CURRENT_MEDICATION_NAME']

54                      ibuprofen; naproxen
322    dextroamphetamine; divalproex sodium
330                sertraline; lansoprazole
363                         methylphenidate
Name: CURRENT_MEDICATION_NAME, dtype: object

In [8]:
non_hps_asd_df = pheno_valid.iloc[non_hps_asd_idx]
len(non_hps_asd_df)

203

In [9]:
non_hps_asd_df[non_hps_asd_df.CURRENT_MED_STATUS == 'taking medication']['CURRENT_MEDICATION_NAME']

2                            aripiprazole; clomipramine 
4      aripiprazole; methylphenidate; lorazepam; meth...
31               methyphenidate; multivitamins; omega 3 
32                           lisdexamfetamine dimesylate
34                               atomoxetine; loratadine
35                      methylphenidate extended release
41                                           atomoxetine
44                               somatropin; atomoxetine
50                      methylphenidate extended release
55           cetirizine hydrochlorid; montelukast sodium
57     fluoxetine; iron supplement; multivitamin; flu...
58                                fluticasone propionate
59                      multivitamin; omega 3 supplement
62                      methylphenidate extended release
64     methylphenidate extended release; multivitamin...
77     methylphenidate hydrochloride; citalopram hydr...
79     amphetamine and dextroamphetamine salts; sertr...
80                             

## Conduct Fisher's exact test to compare proportions between groups

In [10]:
# Count individuals taking medication
n_hps_asd_med = np.where(hps_asd_df['CURRENT_MED_STATUS'] == 'taking medication')[0].size
n_non_hps_asd_med = np.where(non_hps_asd_df['CURRENT_MED_STATUS'] == 'taking medication')[0].size

In [11]:
# Count individuals not taking medication
n_hps_asd_no_med = len(hps_asd_df) - n_hps_asd_med
n_non_hps_asd_no_med = len(non_hps_asd_df) - n_non_hps_asd_med

In [12]:
# Construct the contingency table
contingency_table = np.array([[n_hps_asd_med, n_hps_asd_no_med],
                              [n_non_hps_asd_med, n_non_hps_asd_no_med]])

In [13]:
contingency_table

array([[  4,   5],
       [ 53, 150]])

In [14]:
result = stats.fisher_exact(contingency_table)

In [15]:
result.pvalue

np.float64(0.2544348696197456)